In [ ]:
import openai

from langchain_openai import ChatOpenAI

from ragas import evaluate
from ragas import EvaluationDataset
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextPrecisionWithoutReference

from lupai_mw.db import MongoConnector

In [ ]:
llm = ChatOpenAI(model="gpt-4o")
openai_client = openai.OpenAI()

In [ ]:
mongo_connector = MongoConnector()
assistant_states = [
    state
    async for state in mongo_connector.find_multiple(
        collection="states",
        query_filter={"$expr": {"$gte": [{"$size": "$relevant_chunks"}, 1]}},
    )
]

evaluation_dataset = EvaluationDataset.from_list(
    [
        {
            "user_input": state["query"],
            "retrieved_contexts": [
                rc["text"] for rc in state["relevant_chunks"]
            ],
            "response": state["assistant_response"],
        }
        for state in assistant_states
    ]
)


evaluation_dataset

In [ ]:
evaluator_llm = LangchainLLMWrapper(llm)

In [ ]:
result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextPrecisionWithoutReference()],
    llm=evaluator_llm,
)

In [ ]:
result